# File Inspection (Experimental)

**by Josh Dillon, Tyler Cox, and Aaron Parsons**, last updated August 15, 2022

In [1]:
import numpy as np
import os
import copy
import matplotlib
import matplotlib.pyplot as plt
from uvtools.plot import plot_antpos, plot_antclass
from hera_qm import ant_metrics, ant_class
from hera_cal import io, utils, redcal
from IPython.display import display, HTML
import linsolve
display(HTML("<style>.container { width:100% !important; }</style>"))
_ = np.seterr(all='ignore')  # get rid of red warnings

ImportError: /lib/x86_64-linux-gnu/libz.so.1: version `ZLIB_1.2.9' not found (required by /home/obs/anaconda/envs/RTP/lib/python3.9/site-packages/matplotlib/../../.././libpng16.so.16)

## Parse inputs

To use this notebook interactively, you will have to provide a sum filename path if none exists as an environment variable. All other parameters have reasonable default values.


In [2]:
# get file names
SUM_FILE = os.environ.get("SUM_FILE", None)
# SUM_FILE = '/mnt/sn1/zen.2459797.30001.sum.uvh5'  # If sum_file is not defined in the environment variables, define it here.
DIFF_FILE = SUM_FILE.replace('sum', 'diff')
PLOT = os.environ.get("PLOT", "TRUE").upper() == "TRUE"
if PLOT:
    %matplotlib inline
print(f"SUM_FILE = '{SUM_FILE}'")

ImportError: /lib/x86_64-linux-gnu/libz.so.1: version `ZLIB_1.2.9' not found (required by /home/obs/anaconda/envs/RTP/lib/python3.9/site-packages/matplotlib/../../.././libpng16.so.16)

### Parse Bounds

In [3]:
# ant_metrics bounds for low correlation / dead antennas
AM_CORR_BAD = float(os.environ.get("AM_CORR_BAD", 0.3))
AM_CORR_SUSPECT = float(os.environ.get("AM_CORR_SUSPECT", 0.5))
am_corr_bad = (0, AM_CORR_BAD)
am_corr_suspect = (AM_CORR_BAD, AM_CORR_SUSPECT)

# ant_metrics bounds for cross-polarized antennas
AM_XPOL_BAD = float(os.environ.get("AM_XPOL_BAD", -0.1))
AM_XPOL_SUSPECT = float(os.environ.get("AM_XPOL_SUSPECT", 0))
am_xpol_bad = (-1, AM_XPOL_BAD)
am_xpol_suspect = (AM_XPOL_BAD, AM_XPOL_SUSPECT)

# bounds on autocorrelation power
AUTO_POWER_GOOD_LOW = float(os.environ.get("AUTO_POWER_GOOD_LOW", 5))
AUTO_POWER_GOOD_HIGH = float(os.environ.get("AUTO_POWER_GOOD_HIGH", 30))
auto_power_good = (AUTO_POWER_GOOD_LOW, AUTO_POWER_GOOD_HIGH)
AUTO_POWER_SUSPECT_LOW = float(os.environ.get("AUTO_POWER_SUSPECT_LOW", 1))
AUTO_POWER_SUSPECT_HIGH = float(os.environ.get("AUTO_POWER_SUSPECT_HIGH", 80))
auto_power_suspect = (AUTO_POWER_SUSPECT_LOW, AUTO_POWER_SUSPECT_HIGH)

# bounds on autocorrelation slope
AUTO_SLOPE_GOOD_LOW = float(os.environ.get("AUTO_SLOPE_GOOD_LOW", -0.2))
AUTO_SLOPE_GOOD_HIGH = float(os.environ.get("AUTO_SLOPE_GOOD_HIGH", 0.2))
auto_slope_good = (AUTO_SLOPE_GOOD_LOW, AUTO_SLOPE_GOOD_HIGH)
AUTO_SLOPE_SUSPECT_LOW = float(os.environ.get("AUTO_SLOPE_SUSPECT_LOW", -0.4))
AUTO_SLOPE_SUSPECT_HIGH = float(os.environ.get("AUTO_SLOPE_SUSPECT_HIGH", 0.4))
auto_slope_suspect = (AUTO_SLOPE_SUSPECT_LOW, AUTO_SLOPE_SUSPECT_HIGH)

for bound in ['am_corr_bad', 'am_corr_suspect', 'am_xpol_bad', 'am_xpol_suspect', 
             'auto_power_good', 'auto_power_suspect', 'auto_slope_good', 'auto_slope_suspect']:
    print(f'{bound} = {eval(bound)}')

am_corr_bad = (0, 0.2)
am_corr_suspect = (0.2, 0.4)
am_xpol_bad = (-1, -0.1)
am_xpol_suspect = (-0.1, 0.0)
auto_power_good = (5.0, 30.0)
auto_power_suspect = (1.0, 80.0)
auto_slope_good = (-0.2, 0.2)
auto_slope_suspect = (-0.4, 0.4)


## Load sum and diff data

In [4]:
hd = io.HERADataFastReader(SUM_FILE)
data, _, _ = hd.read(read_flags=False, read_nsamples=False)
hd_diff = io.HERADataFastReader(DIFF_FILE)
diff_data, _, _ = hd_diff.read(read_flags=False, read_nsamples=False)

NameError: name 'io' is not defined

In [5]:
ants = sorted(set([ant for bl in hd.bls for ant in utils.split_bl(bl)]))
auto_bls = [bl for bl in data if (bl[0] == bl[1]) and (utils.split_pol(bl[2])[0] == utils.split_pol(bl[2])[1])]
antpols = sorted(set([ant[1] for ant in ants]))

NameError: name 'hd' is not defined

In [6]:
print(f'File: {SUM_FILE}')
print(f'JDs: {hd.times} ({np.median(np.diff(hd.times)) * 24 * 3600:.5f} s integrations)')
print(f'LSTS: {hd.lsts * 12 / np.pi } hours')
print(f'Frequencies: {len(hd.freqs)} {np.median(np.diff(hd.freqs)) / 1e6:.5f} MHz channels from {hd.freqs[0] / 1e6:.5f} to {hd.freqs[-1] / 1e6:.5f} MHz')
print(f'Antennas: {len(hd.data_ants)}')
print(f'Polarizations: {hd.pols}')

File: /mnt/sn1/2459820/zen.2459820.25314.sum.uvh5


NameError: name 'hd' is not defined

## Find good, suspect, and bad antpols

### Run `ant_metrics`

In [7]:
am = ant_metrics.AntennaMetrics(SUM_FILE, DIFF_FILE, sum_data=data, diff_data=diff_data)
am.iterative_antenna_metrics_and_flagging(crossCut=am_xpol_bad[1], deadCut=am_corr_bad[1])

NameError: name 'ant_metrics' is not defined

In [8]:
totally_dead_ants = [ant for ant, i in am.removal_iteration.items() if i == -1]
am_totally_dead = ant_class.AntennaClassification(good=[ant for ant in ants if ant not in totally_dead_ants], bad=totally_dead_ants)
am_corr = ant_class.antenna_bounds_checker(am.final_metrics['corr'], bad=[am_corr_bad], suspect=[am_corr_suspect], good=[(0, 1)])
am_xpol = ant_class.antenna_bounds_checker(am.final_metrics['corrXPol'], bad=[am_xpol_bad], suspect=[am_xpol_suspect], good=[(-1, 1)])
ant_metrics_class = am_totally_dead + am_corr + am_xpol
print('ant_metrics Classification:\n\n', ant_metrics_class)

NameError: name 'am' is not defined

### Examine and classify autocorrelations

In [9]:
auto_power_class = ant_class.auto_power_checker(data, good=auto_power_good, suspect=auto_power_suspect)
auto_slope_class = ant_class.auto_slope_checker(data, good=auto_slope_good, suspect=auto_slope_suspect, edge_cut=100, filt_size=17)
auto_class = auto_power_class + auto_slope_class
print('Autocorrelation Classification:\n\n', auto_class)

NameError: name 'ant_class' is not defined

In [10]:
if PLOT:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=100, sharey=True, gridspec_kw={'wspace': 0})
    labels = []
    colors = ['darkgreen', 'goldenrod', 'maroon']
    for ax, pol in zip(axes, antpols):
        for ant in auto_class.ants:
            if ant[1] == pol:
                color = colors[auto_class.quality_classes.index(auto_class[ant])]
#                 ax.semilogy(hd.freqs / 1e6, np.mean(data[utils.join_bl(ant, ant)], axis=0), color=color, lw=.5)
                ax.semilogy(np.mean(data[utils.join_bl(ant, ant)], axis=0), color=color, lw=.5)
#         ax.set_xlabel('Frequency (MHz)', fontsize=12)
        ax.set_xlabel('Channel', fontsize=12)

    axes[0].set_ylabel('Raw Autocorrelation', fontsize=12)
    axes[1].legend([matplotlib.lines.Line2D([0], [0], color=color) for color in colors], 
                   [cls.capitalize() for cls in auto_class.quality_classes], ncol=1, fontsize=12, loc='upper right', framealpha=1)
    plt.tight_layout()

NameError: name 'plt' is not defined

### Show antenna classification summary array plot

In [11]:
final_class = auto_class + ant_metrics_class
if PLOT:
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), dpi=100, gridspec_kw={'width_ratios': [2, 1]})
    plot_antclass(hd.antpos, final_class, ax=axes[0], ants=[ant for ant in hd.data_ants if ant < 320], legend=False, title='HERA Core')
    plot_antclass(hd.antpos, final_class, ax=axes[1], ants=[ant for ant in hd.data_ants if ant >= 320], radius=50, title='Outriggers')

NameError: name 'auto_class' is not defined

## Metadata

In [12]:
from hera_cal import __version__
print('hera_cal:', __version__)
from hera_qm import __version__
print('hera_qm:', __version__)

hera_cal: 3.1.4.dev3+g68bd8c3
hera_qm: 2.0.3.dev44+g7d4aa18
